# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План проекта:**
- определить и заполнить пропущенные значения:
- описать, какие пропущенные значения были обнаружили;
- привести возможные причины появления пропусков в данных;
- объяснить, по какому принципу заполнены пропуски;
- заменить вещественный тип данных на целочисленный:
- пояснить, как выбиран метод для изменения типа данных;
- удалить дубликаты:
- пояснить, как выбиран метод для поиска и удаления дубликатов в данных;
- привести возможные причины появления дубликатов;
- выделить леммы в значениях столбца с целями получения кредита:
- описать, как проводена лемматизация целей кредита;
- категоризировать данные:
- перечислить, какие «словари» выделили для этого набора данных, и объясните, почему.

**По результату исследования нужно ответить на вопросы:**
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

**Описание данных:**

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Изучение общей информации

In [18]:
#импортируем библиотеки
import pandas as pd
from collections import Counter
from pymystem3 import Mystem
m = Mystem() 

In [19]:
df = pd.read_csv('data.csv')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [20]:
#смотрим информацию о таблице, где есть пропуски, где какой тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Есть пропуски в столбцах "days_employed" и "total_income". Либо их не заполняли, чаще всего это данные связаны. Либо это ошибка

Стобец "children" имеет определенные значения и отвечает за количество детей. Есть значения -1 и 20 их заменим на медианные.

Есть значение 0 в "dob_years", его либо не внесли или ошибка. В любом случае заменим медианной.

"days_employed" у большинства имеет отрицательное значение, скорее всего это ошибка и был взято противополжное значение. возьмем модуль и поправим это. У пенсионеров слишком большие значения, с чем это связанно непонятно.

В "education" и "purpose" есть явные и неявные дубликаты.

В "education" и "family_status" есть соответствия с категориями из "education_id" и "family_status_id". Надо получить таблицу соотношений, чтобы понимать, что есть что.

## Предобработка данных

**Обработка пропусков**

In [21]:
#цикл ведет  все уникакльные значения во всех столбцах
for column_name in df.columns: 
    print(df[column_name].unique())

[ 1  0  3  2 -1  4 20  5]
[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
[0 1]
[253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на провед

In [22]:
 #берем модуль из всех значений, т.к имеем отрицательные значения.
df['days_employed'] = abs(df['days_employed'])
#заполняем пропуски медианой всех имеющихся значений
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()) 
df['days_employed'].isna().sum()

0

In [23]:
#тотал инкам тоже заполним медианной, т.к. пропущенные значения в зарплатах можно отлично заменить медианными, 
#чтобы это не отразилось на процентном соотношении
df['total_income'] = df['total_income'].fillna(df['total_income'].median())
df['total_income'].isna().sum()

0

**Вывод**

Пропущенные значения в столбцах ежемесячного заработка и трудового стажа заменили медианными значениями.

**Замена типа данных**

In [24]:
#в тотал_инкам и в days_employed мы имеем дробные значения с большим количесвом чисел после запятой. 
#Если мы округлим нанокопейки до рубля это не скажется на нашем распределении
df['total_income'] = df['total_income'].astype(int)
df['days_employed'] = df['days_employed'].astype(int)

**Обработка дубликатов**

In [25]:
#приведем все к нижнему регистру в столбце education т.к. только там мы имеем дубликаты связанные с регистром
df['education'] = df['education'].str.lower()

In [26]:
#заменим значения -1 и 20 на 1 и 2 с помошью цикла
for row in range(len(df['children'])): 
    if df.loc[row, 'children'] == -1:
        df['children'][row] = 1
    elif df.loc[row, 'children'] == 20:
        df['children'][row] = 2
df['children'].unique()

C:\Users\User\AppData\Local\Temp/ipykernel_19472/2188054563.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['children'][row] = 1
C:\Users\User\AppData\Local\Temp/ipykernel_19472/2188054563.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['children'][row] = 2


array([1, 0, 3, 2, 4, 5], dtype=int64)

**Лемматизация**

In [30]:
#обьединяем в столбец в одну строку
purpose_str = ' '.join(data['purpose']) 
#лемматизируем строку
lemmas = m.lemmatize(purpose_str)
#считаем саммые частые леммы
Counter(lemmas)

Counter({'покупка': 5912,
         ' ': 55201,
         'жилье': 4473,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

**Категоризация данных**

У нас 6 категорий в целях кредита:
- для свадьбы(5),
- для образования(4),
- для машины(3),
- для жилья(2),
- для коммерческой недвижимости(1)
- остальное(0)

6 категорий по наличию детей: от 0 до 5

5 категорий по семмейному положению

5 категорий по уровню дохода. 20% от общей выборки входят в одну категорию

In [32]:
#вводим функцию, которая ищет подходящую лемму и присваивает общую категорию по целям
def categorization_purpose(row): 
    purpose = row['purpose']
    if 'недв' in purpose:
        return 1
    elif 'жиль' in purpose:
        return 2
    elif 'авто' in purpose:
        return 3
    elif 'обра' in purpose:
        return 4
    elif 'сва' in purpose:
        return 5
    return 6
df['purpose_id'] = df.apply(categorization_purpose, axis=1)

In [37]:
#создаем список с названием наших категорий
labels=['низкий', 'ниже среднего', 'средний', 'выше среднего', 'высокий'] 
#создаем столбец 'income_id' в него ставим значения от функции
#'qcut' и делим на 5 ровных частей и присваиваем им наши названия
df['income_id'] = pd.qcut(df['total_income'], 5, labels=labels) 
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21525 non-null  int64   
 1   days_employed     21525 non-null  int32   
 2   dob_years         21525 non-null  int64   
 3   education         21525 non-null  object  
 4   education_id      21525 non-null  int64   
 5   family_status     21525 non-null  object  
 6   family_status_id  21525 non-null  int64   
 7   gender            21525 non-null  object  
 8   income_type       21525 non-null  object  
 9   debt              21525 non-null  int64   
 10  total_income      21525 non-null  int32   
 11  purpose           21525 non-null  object  
 12  purpose_id        21525 non-null  int64   
 13  income_id         21525 non-null  category
dtypes: category(1), int32(2), int64(6), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_id,income_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,2,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,3,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,2,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,4,высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,5,выше среднего


**Вывод**

Привели все данные в порядок. Добавили категории по заработку людей.

## Анализ данных

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [39]:
#создаем сводную таблицу со значениями
#общего количества людей в категории, кол-во людей с задолжностью
data_children = df.pivot_table(index=['children'],values='debt',aggfunc=('sum','count')) 
#добавляем столбец в котором считаем кол-во процентов людей с долгами
data_children['ratio'] = 100*data_children['sum']/data_children['count'] 
data_children

,count,sum,ratio
children,,,
0,14149,1063,7.512898
1,4865,445,9.146968
2,2131,202,9.479118
3,330,27,8.181818
4,41,4,9.756098
5,9,0,0.000000


**Вывод**

Зависимость между кол-м детей и возвратом кредита в срок есть. Люди без детей имеют самый низкий процент долга, а с детьми уже процент выше.

Нет детей - вероятность долга 7,5%

1 ребенок - вероятность долга 9,1%

2 ребенка - вероятность долга 9,4%

3 ребенка - вероятность долга 8,1%

4 ребенка - вероятность долга 9,7%

5 детей - вероятность долга 0%

Можем заметить что, если у человека 5 детей, то вероятность долга нулевая, но в выборке всего 9 человек, поэтому можем пренебречь этой категорией. Тоже самое можно и сказать про людей с 4 детьми(41 человек) у которых процент долга выше, чем у людей с 3 детьми.

Веротяность долга растет с кол-м детей, но только до 3 детей. Если у человека 3 ребенка, вероятность долга ниже. Можно предположить, что и дальше с увеличением детей будет уменьшаться вероятность долга.

- Самые надежные заемщики без детей 7,5% вероятность долга. 
- Самые ненадежные заемщики в 2 детьми 9,4% вероятность долга.

- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [40]:
data_family_status = df.pivot_table(index=['family_status','family_status_id'],values='debt',aggfunc=('sum','count'))
data_family_status['ratio'] = 100*data_family_status['sum']/data_family_status['count']
data_family_status

,,count,sum,ratio
family_status,family_status_id,,,
Не женат / не замужем,4,2813,274,9.740491
в разводе,3,1195,85,7.112971
вдовец / вдова,2,960,63,6.562500
гражданский брак,1,4177,388,9.288963
женат / замужем,0,12380,931,7.520194


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок есть. Одинокие люди являются самыми надежными и ненадежными заемщиками.

- Люди без партнера - вероятность долга почти 10%
- Вдовцы - вероятность долга почти 6,6 %


- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [42]:
data_income = df.pivot_table(index=['income_id'],values='debt',aggfunc=('sum','count'))
data_income['ratio'] = 100*data_income['sum']/data_income['count']
data_income

,count,sum,ratio
income_id,,,
низкий,4305,345,8.013937
ниже среднего,4305,357,8.292683
средний,4305,364,8.455285
выше среднего,4305,372,8.641115
высокий,4305,303,7.038328


**Вывод**

Зависимость между уровнем дохода и возвратом кредита в срок есть. При доходе ниже высокого вероятность долга меняется не критично. 

- Люди с высоким доходом возвращают кредит на 1% чаще всех остальных.

- **Как разные цели кредита влияют на его возврат в срок?**

In [43]:
data_purpose = df.pivot_table(index=['purpose_id'],values='debt',aggfunc=('sum','count'))
data_purpose['ratio'] = 100*data_purpose['sum']/data_purpose['count']
data_purpose

,count,sum,ratio
purpose_id,,,
1,6367,474,7.444636
2,4473,308,6.885759
3,4315,403,9.339513
4,4022,370,9.199403
5,2348,186,7.921635


**Вывод**

Зависимость между целью кредита и возвратом кредита в срок есть. 


- Самый надежные заемщики, те кто берет какую-либо недвижимость в кредит. Коммерческая или некоммерческая, 7,4% и 6,8% вероятность долга, соотвественно.
- Самые ненадежные кредиты, кредиты на автомобиль и образование. 9,3% и 9,2% вероятность долга, соотвественно.

## Вывод

Мы нашли зависимость вероятности долга от 4 параметров. И можем помтроить портрет надежного и ненадежного заемщика.
- Самый надежный заемщик банка: Вдовец без детей с высоким уровня дохода с кредитом на недвижимость
- Самый ненадежный заемщик банка: Неженатый человек с 2 детьми с доходом ниже высокого с кредитом на машину или образование.